# Yolo Mean Average Precision
The purpose of this notebook is to calculate mAP for each of the YOLOv8 models. The mAP@[0.5:0.95:0.05] will be calculated for each class and the results will be saved to .csv file

In [3]:
import commonPaths
import commonCocoPreprocessingFunctions as preprocFuncs
import ScoreCalculator
from importlib import reload
reload(commonPaths)
reload(preprocFuncs)
reload(ScoreCalculator)
from ultralytics import YOLO
import os
import json
import pandas as pd
import math as m
from tqdm import tqdm

In [5]:
def reshapeGroundTruthBoxesForImg(annotYoloJSONFilePath):
    '''
        ### reshapeGroundTruthBoxesForImg
        :param annotYoloJSONFilePath: Path to JSON file containing GroundTruth Boxes in the YOLO format
        (the file produced by the createAnnotJSONForYolo function). The format should be like this:
        {"imageId": [ {"yoloCatId": ycId, "bbox": [xMid, yMid, width, height]}, {(...)anotherAnnot(...)} ]}
        :return: a list containing groundTruth boxes for image. Each element in a list is a list of groundTruth
        boxes for the given image (written in the YOLO format [imgId, class, xMid, yMid, width, height]).
        It is not a dictionary since the results in the results given by YOLO models are in the same order
        as files in the directory.
    '''
    gtsFile= open(annotYoloJSONFilePath)
    groundTruthsJSON = json.load(gtsFile)
    groundTruths = []
    for imgId, annots in groundTruthsJSON.items():
        groundTruthsForImg = []
        for ann in annots:
            gtClass = ann["yoloCatId"]
            bbox = ann["bbox"]
            groundTruthBox = [int(imgId), gtClass, bbox[0], bbox[1], bbox[2], bbox[3]]
            groundTruthsForImg.append(groundTruthBox)
        groundTruths.append(groundTruthsForImg)
    return groundTruths

In [6]:
def createDataFrameHeader(annotDir, annotFileName):
    '''
        ### createDataFrameHeader
        creates column names for data frames used to store calculated APs for each batch for each class.

        :param annotDir: directory with annotations for the given dataset
        :param annotFileName: file name with annotations in the given directory (used to retrieve info about classes)
        :return: a list of column names corresponding to the classes within the dataset
    '''
    instancesJSON = preprocFuncs.getInstancesAsJSON(annotDir, annotFileName)
    categoryIdToNameAndYoloId = preprocFuncs.associateCategoryIdWithItsNameAndYoloId(instancesJSON)
    columnNames = []
    for cat in categoryIdToNameAndYoloId.values():
        columnNames.append(f"{cat.categoryName} {cat.yoloId}".replace(" ", "_"))
    return columnNames

In [7]:
def calculateIoUThresholds(minIoU, maxIoU, stepIoU):
    '''
        ### calculateIoUThresholds
        creates a range of IoU thresholds from the given min, max and step IoU

        :param minIoU: minimum threshold for which to start calculating
        :param maxIoU: maximum threshold for which to stop calculating
        :param stepIoU: step for how much to increase IoU threshold in each iteration
        :return: list containing all IoU thresholds for which to calculate APs
    '''
    iouThresholds = []
    for i in range( int(minIoU*100), int((maxIoU+stepIoU)*100), int(stepIoU*100)):
        iouThresholds.append(i/100)
    return iouThresholds

In [8]:
def predictResultsAndCalculatemAPForEachClass(model, pathsToImgs, batchSize, columnNames, groundTruths, iouThresholds):
    '''
        ### predictResultsAndCalculatemAPForEachClass
        The function uses the model to predict bboxes and then calculates mAP for each class. 
        mAP is calculated for IoU thresholds from the range given by the params. The mAP is calculated for
        several batches since not dividing in batches is difficult computationally. The function returns
        calculated mAP for each class. To calculate final mAP one would have to average the results for the number
        of classes.

        :param model: YOLO model on which predictions should be made
        :param pathsToImgs: a list with paths to imgs. (e.g. result of preprocFuncs.providePathsToImages)
        :param batchSize: The size of a batch for dividing the dataset in smaller parts.
        :param columnNames: a list of column names for the output series. (e.g. result of createDataFrameHeader)
        :param groundTruths: list of list of groundTruth boxes for each Image. Boxes should be in the YOLO format
        (e.g. result of reshapeGroundTruthBoxesForImg)
        :param iouThresholds: a list of IoU thresholds for which to calculate APs (e.g. result of calculateIoUThresholds)\
        
        :return: a data series with mean Average Precisions for each class.
    '''
    noBatches = m.ceil(len(pathsToImgs) / batchSize)
    averagePrecisionsForIOUThreshold = []
    for i in iouThresholds:
        averagePrecisionsForIOUThreshold.append(pd.DataFrame(columns=columnNames))

    for i in tqdm(range(0,noBatches)):
        start = i * batchSize
        end = (start + batchSize) if (start + batchSize) < len(pathsToImgs) else len(pathsToImgs)
        results = model.predict(pathsToImgs[start:end], verbose=False, device="0")
        predictedBoxes = []
        for result in results:
            imgId = int(result.path.split("/")[-1].split(".")[0])
            for bbox in result.boxes:
                predClass = int(bbox.cls)
                predConf = round(float(bbox.conf),6)
                predBbox = bbox.xywhn
                x = round(predBbox[0][0].item(),6)
                y = round(predBbox[0][1].item(),6)
                width = round(predBbox[0][2].item(),6)
                height = round(predBbox[0][3].item(),6)
                predictedBox = [imgId, predClass, x, y, width, height, predConf]
                predictedBoxes.append(predictedBox)
        groundTruthsToPass = []
        for gtList in groundTruths[start:end]:
            groundTruthsToPass.extend(gtList)

        for j, avgPrecisionForBatch in enumerate(averagePrecisionsForIOUThreshold):
            iouThresh = iouThresholds[j]
            avgPrecisionForBatch.loc[len(avgPrecisionForBatch.index)] = ScoreCalculator.meanAveragePrecission(predictedBoxes, groundTruthsToPass, iouThreshold=iouThresh)

    apsForClasses = pd.DataFrame(columns=columnNames)
    for frame in averagePrecisionsForIOUThreshold:
        apsForClasses.loc[len(apsForClasses.index)] = frame.sum(axis=0)
    mapsForClasses = (apsForClasses.sum() / (noBatches * len(averagePrecisionsForIOUThreshold) ))
    return mapsForClasses

In [9]:
def saveResults(calculatedMAPsForEachClass, modelName, batchSize, minIoU, maxIoU, stepIoU, pathToOutFile):
    '''
        ### saveResults
        saves results in the given directory. 
        The file name is mAPResults_{batchSize}_{minIoU}_{maxIoU}_{stepIoU}_{modelName}.csv

        :param calculatedMAPsForEachClass: mAPs for each class (e.g. result of predictResultsAndCalculatemAPForEachClass)
        :param modelName: name of the model
        :param batchSize: The size of a batch for dividing the dataset in smaller parts.
        :param minIoU: minimum threshold for which to start calculating
        :param maxIoU: maximum threshold for which to stop calculating
        :param stepIoU: step for how much to increase IoU threshold in each iteration
        :param pathToOutFile: the directory where to write file (with trailing slash)
    '''
    outPath = pathToOutFile
    outPath+=f"MAPResults_{batchSize}_{minIoU}_{maxIoU}_{stepIoU}_{modelName}.csv"
    if(os.path.isfile(outPath)):
            msg = f"Results file already exists at \"{outPath}\""
            raise Exception(msg)
    calculatedMAPsForEachClass.to_csv(outPath, header=False)

In [13]:
def calculateMeanAvgPrecisionsForClassForModel(model, batchSize, minIoU, maxIoU, stepIoU, isTrain=False):
    '''
        ### calculateMeanAvgPrecisionsForClassForModel
        function to calculate mean average precisions for each class for the given model.

        :param model: YOLO model on which predictions should be made
        :param batchSize: The size of a batch for dividing the dataset in smaller parts.
        :param minIoU: minimum threshold for which to start calculating
        :param maxIoU: maximum threshold for which to stop calculating
        :param stepIoU: step for how much to increase IoU threshold in each iteration
        :param isTrain: whether to calculate mAP on train or val dataset. (Set to False by default)

        :return: a data series with mean Average Precisions for each class.
    '''
    pathsDict = preprocFuncs.providePaths(isTrain)
    pathsToImgs = preprocFuncs.providePathsToImages(pathsDict["COCO_IMG_DIR"])
    groundTruths = reshapeGroundTruthBoxesForImg(pathsDict["ANNOT_YOLO_JSON_FILE"])
    columnNames = createDataFrameHeader(pathsDict["COCO_ANNOT_DIR"], pathsDict["ANNOT_FILENAME"])
    iouThresholds = calculateIoUThresholds(minIoU, maxIoU, stepIoU)
    calculatedMAPsForEachClass = predictResultsAndCalculatemAPForEachClass(model, pathsToImgs, batchSize, columnNames, groundTruths, iouThresholds)
    saveResults(calculatedMAPsForEachClass.astype(float), model.model_name, batchSize, minIoU, maxIoU, stepIoU, pathsDict["MAP_RESULTS_DIR"])
    return calculatedMAPsForEachClass
    

In [11]:
model = YOLO('yolov8n.pt').to('cuda')

In [12]:
calculateMeanAvgPrecisionsForClassForModel(model, 100, 0.5, 0.95, 0.05)

100%|██████████| 50/50 [05:08<00:00,  6.18s/it]


person_0         tensor(0.4494)
bicycle_1        tensor(0.3154)
car_2            tensor(0.3181)
motorcycle_3     tensor(0.3802)
airplane_4       tensor(0.6797)
                      ...      
vase_75          tensor(0.3334)
scissors_76      tensor(0.7478)
teddy_bear_77    tensor(0.4649)
hair_drier_78    tensor(0.8400)
toothbrush_79    tensor(0.5053)
Length: 80, dtype: object